In [2]:
import gate
import numpy as np
%load_ext autoreload
%autoreload 2
init = [1, 0, 0, 0]
print(gate.gate['CX'] @ (np.kron(gate.gate['H'],gate.gate['I']) @ init))

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [10]:
for i in range(0, 10):
    k = np.kron(gate.gate['I'],gate.gate['H'])


In [ ]:
for i in range(0, 1000000):
    k = gate.gate['IH']

In [6]:
A = np.random.rand(2**15, 2**15)
A = A.astype(np.complex128)
B = np.array([[1,1],[1,1]])

In [56]:
for i in range(1):
    k = np.kron(A, np.array([[0,0],[0,1]]))

In [7]:
A @ A

In [65]:
import tensor
k1 = tensor.ARY(A, 1)
print(k1.shape)

(2097152, 80)


In [52]:
import numpy as np

def AP3(A):
    rows, cols = A.shape
    kron_result = np.zeros((2 * rows, 2 * cols))
    kron_result[1::2, 1::2] = A
    return kron_result

# Example usage
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
result = AP3(A)
print(result)


[[0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 2. 0. 3.]
 [0. 0. 0. 0. 0. 0.]
 [0. 4. 0. 5. 0. 6.]
 [0. 0. 0. 0. 0. 0.]
 [0. 7. 0. 8. 0. 9.]]
